In [1]:
import numpy as np
import findspark
findspark.init()
import pyspark as ps
import re
#from pyspark.ml.feature import Tokenizer, RegexTokenizer
import string
from pyspark import SparkContext
from pyspark.mllib.feature import HashingTF, IDF

In [2]:
#import findspark
#findspark.init("/usr/local/Cellar/apache-spark/2.3.1/bin")

In [3]:
def clean_text(text):
  text = lower(text)
  text = regexp_replace(text, "^rt ", "")
  text = regexp_replace(text, "(https?\://)\S+", "")
  text = regexp_replace(text, "[^a-zA-Z0-9\\s]", "")
  return text

In [4]:
def remove_punctuation(text):
    """This method removes the punctuation like commas and quotes from the text (string). We also want to keep contractions together. The method also make the words lower cased."""
    # split into words by white space
    words = text.split()
    words_lower = [w.lower() for w in words]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words_lower]
    return stripped

In [ ]:
def tf(word, text):
    """Function that calculates tf for a given word in a list of words (text)"""
    total = len(text)
    word_count = sum([x==word for x in text])
    tf_val = word_count/total
    return tf_val

In [ ]:
def idf(text, document):

In [5]:
def paper_recommender(paper_in_PMID, text_rdd, n):
    """The main function that gets a PMID for a paper, and recommends n similar papers, using the metadata of all papers in text_rdd"""
    # Split text by tab
    text_rdd_n = text_rdd.map(lambda x: x.split('\t'))
    # Separate the header from the text file
    header = text_rdd_n.first()
    # Remove the header from the text_rdd
    text_rdd_n = text_rdd_n.filter(lambda x: x != header)
    # Extract title and abstract for each paper and remove punctuations from them
    print("Cleaning the Title & Abstract...")
    paper_abs_title_rdd = text_rdd_n.map(lambda x: (remove_punctuation(x[1]), remove_punctuation(x[5])))
    print(text_rdd_n.first())   
    print("----------------")
    print(paper_abs_title_rdd.first()) 
    
#     hashingTF = HashingTF()
#     tf = hashingTF.transform(text_rdd_n)
    
#     # While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
#     # First to compute the IDF vector and second to scale the term frequencies by IDF.
#     tf.cache()
#     idf = IDF().fit(tf)
#     tfidf = idf.transform(tf)
    
#     print("*********************")
#     print(tfidf)

    

    return(paper_abs_title_rdd )   

In [6]:
if __name__ == '__main__':
    #sc.stop()
    sc = ps.SparkContext('local[4]')
    text_rdd = sc.textFile('./data.tab')
    paper_recommender(27000000, text_rdd, 10)    

Cleaning the Title & Abstract...
['27990000', 'Phenotypic difference between ?(srl-recA)306 and ?recA::Km elucidated by next-generation sequencing combined with a long-PCR system.', "['Suzuki', 'Kaidow', 'Meya', 'Masuya', 'Shiina']", '2017', 'The Journal of general and applied microbiology', 'Many significant gene mutations in E. coli have contributed to the development of genetics. Among these, a commonly used recA mutation, ?(srl-recA)306 has been sequenced by a next-generation sequencer combined with a long PCR. An original report described that ?(srl-recA)306 cells were deleted from srlR to recA genes in their genome. The next-generation sequencer enables more accurate details to be determined. We ask whether both surrounding genes from hypF to norV for srlR and alaS for recA is there first. The long PCR was carried out with primers, norR and alaS, and amplified DNA fragments differed in length from wild to ?(srl-recA)306 cells, suggesting that an entire ?(srl-recA)306 mutation was

Py4JJavaError: An error occurred while calling z:org.apache.spark.mllib.api.python.SerDe.pythonToJava.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.collec

In [15]:
text = ['a', 'bb', 'c', 'd', 'a']
print(sum([x=='a' for x in text]))

2
